In [9]:
import P_Cyg as pc
import numpy as np
import pandas as pd
from glob import glob
from astropy.time import Time

In [10]:
find_detector = lambda files : [f[4:7]    for f in files]
find_date = lambda files : [f[14:18]+'-'+f[18:20]+'-'+f[20:22]    for f in files]
find_odate = lambda files : [f[63:67]+'-'+f[67:69]+'-'+f[69:71]    for f in files]
sig = lambda x : float("{0:.3f}".format(x))

# H$\alpha$

### Observed

In [11]:
datadir = 'C:/Users/Keyan/Desktop/Science/Data/P-Cygni/Data/'
rtxt = glob(datadir + 'hpolccd*r_hw.fits.txt')
btxt = glob(datadir + 'hpolccd*b_hw.fits.txt')
rettxt = glob(datadir + '*ret*.txt')
ccdtxt = []
for rf in rtxt:
    for bf in btxt:
        if rf[63:71]==bf[63:71]:
            ccdtxt.append(np.array([rf,bf]))
ccddfs = [[pd.read_csv(f[0],delim_whitespace=True,header=0,names=["Wavelength","Flux","Q","U","Error"]),pd.read_csv(f[1],delim_whitespace=True,header=0,names=["Wavelength","Flux","Q","U","Error"])]    for f in ccdtxt]
ccddfs = [pd.concat([p[0],p[1][p[1].Wavelength < 6100]]).sort_values("Wavelength")    for p in ccddfs]
retdfs = [pd.read_csv(f,delim_whitespace=True,header=0,names=["Wavelength","Flux","Q","U","Error"])    for f in rettxt]
obdfs = retdfs + ccddfs

In [12]:
hmask = lambda df: df[(df.Wavelength>6540) & (df.Wavelength<6600)]
ohdfs = [hmask(df)    for df in obdfs]

In [13]:
ohq,ohu = [sig(sum(df.Q*(df.Error**-2))/sum(df.Error**-2))    for df in ohdfs],[sig(sum(df.U*(df.Error**-2))/sum(df.Error**-2))    for df in ohdfs]

### Intrinsic

In [14]:
intrinsic_dir = "C:/Users/Keyan/Desktop/Science/Data/P-Cygni/Data/intrinsic_pols/"
files = glob(intrinsic_dir+"*")
retfiles = [f    for f in files if f[69]=='o']
rfiles = [f    for f in files if f[69]=='r']
bfiles = [f    for f in files if f[69]=='b']
matched = []
for rf in rfiles:
    for bf in bfiles:
        if rf[64:69]==bf[64:69]:
            matched.append(np.array([rf,bf]))
matched = np.array(matched)
mdfs = [[pd.read_csv(f[0]),pd.read_csv(f[1])]    for f in matched]
mdfs = [pd.concat([p[0],p[1][p[1].Wavelength < 6100]]).sort_values("Wavelength")    for p in mdfs]
rdfs = [pd.read_csv(f)    for f in files if f[72]=='r']
bdfs = [pd.read_csv(f)    for f in files if f[72]=='b']
retdfs = [pd.read_csv(f)     for f in retfiles]
alldfs = retdfs+mdfs
t = [float(f[64:69])    for f in retfiles] + [float(f[0][64:69])    for f in matched]
t = [round(x)    for x in t]
od = find_odate(rettxt + list(np.array(ccdtxt)[:,0]))
ot = Time(list(od),format='fits',location=(43,-89,0)).mjd

In [15]:
ihdfs = [hmask(df)    for df in alldfs]

In [16]:
ihq,ihu = [sig(sum(df.Q*(df.Error**-2))/sum(df.Error**-2))    for df in ihdfs],[sig(sum(df.U*(df.Error**-2))/sum(df.Error**-2))    for df in ihdfs]
he = [sig(np.sqrt(np.sum(df.Error**2))/np.sqrt(df.Error.size))    for df in ihdfs]

## Pfew

In [18]:
pdir = datadir+"synthetic_filters/"
pccd = pd.read_csv(pdir+"pfew_ccd.txt",delim_whitespace=True,names=["File/Night","q","u","Error"],header=0)
pret = pd.read_csv(pdir+"pfew_ret.txt",delim_whitespace=True,names=["File/Night","q","u","Error"],header=0)
pfew = pd.concat([pret,pccd]).reset_index().drop(columns='index').astype({"q":np.float64,"u":np.float64, "Error":np.float64})
pq = [sig(q)    for q in pfew.q]
pu = [sig(u)    for u in pfew.u]
pe = [sig(e)    for e in pfew.Error]

### H table

In [19]:
htable = pd.DataFrame({"MJD":t,
                       "Date":od,
                       "oq":ohq,
                       "ou":ohu,
                       "iq":ihq,
                       "iu":ihu,
                       "Error":he,
                       "pq":pq,
                       "pu":pu,
                       "pe":pe},index=None)

# $V$-Band

### Observed

In [20]:
ccd_dir = 'C:/Users/Keyan/Desktop/Science/Data/P-Cygni/Data/synthetic_filters/VBand_CCD_ISP.txt'
ret_dir = 'C:/Users/Keyan/Desktop/Science/Data/P-Cygni/Data/synthetic_filters/VBand_Reticon_ISP.txt'

In [21]:
ccd = pd.read_csv(ccd_dir,delim_whitespace=True,names=["File/Night","q","u","Internal_Error","Systematic_Error"])
ret = pd.read_csv(ret_dir,delim_whitespace=True,names=["File/Night","q","u","Internal_Error","Systematic_Error"])
vtable = pd.concat([ret,ccd]).reset_index().drop(columns='index')
vtable["Detector"] = find_detector(vtable["File/Night"])
vtable["Date"] = find_date(vtable["File/Night"])
vtable["MJD"] = Time(list(vtable.Date.values),format='fits',location=(43,-89,0)).mjd

In [22]:
vtable["Internal_Error"] = [sig(x)    for x in vtable["Internal_Error"]]
vtable["Systematic_Error"] = [sig(x)    for x in vtable["Systematic_Error"]]
vtable.q = [sig(x)    for x in vtable.q]
vtable.u = [sig(x)    for x in vtable.u]

### Intrinsic

In [23]:
ccddf = pd.read_csv('C:/Users/Keyan/Desktop/Science/Data/P-Cygni/Data/synthetic_filters/VBand_CCD_Int.txt',
                delim_whitespace=True,names=["File/Night","q","u","Internal_Error","Systematic_Error"])
retdf = pd.read_csv('C:/Users/Keyan/Desktop/Science/Data/P-Cygni/Data/synthetic_filters/VBand_Reticon_Int.txt',
                delim_whitespace=True,names=["File/Night","q","u","Internal_Error","Systematic_Error"])

In [24]:
intvtable = pd.concat([retdf,ccddf]).reset_index().drop(columns='index')
intvtable["Detector"] = find_detector(intvtable["File/Night"])
intvtable["Date"] = find_date(intvtable["File/Night"])
intvtable["MJD"] = Time(list(intvtable.Date.values),format='fits',location=(43,-89,0)).mjd
intvtable["Internal_Error"] = [sig(x)    for x in intvtable["Internal_Error"]]
intvtable["Systematic_Error"] = [sig(x)    for x in intvtable["Systematic_Error"]]
intvtable.q = [sig(x)    for x in intvtable.q]
intvtable.u = [sig(x)    for x in intvtable.u]

### Vtable

In [25]:
V_table = pd.DataFrame({"Detector":vtable.Detector,
                        "MJD":vtable.MJD,
                        "Date":vtable.Date,
                        "oq":vtable.q,
                        "ou":vtable.u,
                        "Systematic Error":vtable.Systematic_Error,
                        "Internal Error":vtable.Internal_Error,
                        "iq":intvtable.q,
                        "iu":intvtable.u,
                        "intSystematic Error":intvtable.Systematic_Error,
                        "intInternal Error":intvtable.Internal_Error},index=None)

# Big Table

In [26]:
dformat = lambda dlist: ["Reticon" if d == 'ret' else "CCD"    for d in dlist]
bigtable = pd.DataFrame({"Detector":dformat(vtable.Detector),
                        "MJD":htable.MJD,
                         "Date":htable.Date,
                        "ovq":V_table.oq,
                        "ovu":V_table.ou,
                         "ivq":V_table.iq,
                         "ivu":V_table.iu,
                        "sve":V_table["Systematic Error"],
                         "ive":V_table["Internal Error"],
                        "ohq":htable.oq,
                        "ohu":htable.ou,
                        "ihq":htable.iq,
                        "ihu":htable.iu,
                         "he":htable.Error,
                        "pq":htable.pq,
                        "pu":htable.pu,
                        "pe":htable.pe}, index=None)
bigtable.to_csv("C:/Users/Keyan/Desktop/Science/Data/P-Cygni/Data/bigtable.txt")
print(len(bigtable))
print(bigtable.to_latex(index=False))

76
\begin{tabular}{lrlrrrrrrrrrrrrrr}
\toprule
Detector &    MJD &        Date &    ovq &    ovu &    ivq &    ivu &    sve &    ive &    ohq &    ohu &    ihq &    ihu &     he &     pq &     pu &     pe \\
\midrule
 Reticon &  47668 &  1989-05-22 &  0.305 &  0.997 & -0.134 & -0.135 &  0.020 &  0.004 &  0.336 &  1.005 & -0.057 & -0.018 &  0.133 &  0.046 &  0.027 &  0.172 \\
 Reticon &  47672 &  1989-05-26 &  0.611 &  1.057 &  0.172 & -0.076 &  0.020 &  0.004 &  0.469 &  1.058 &  0.108 &  0.059 &  0.091 & -0.110 &  0.115 &  0.108 \\
 Reticon &  47697 &  1989-06-20 &  0.289 &  1.128 & -0.150 & -0.005 &  0.020 &  0.004 &  0.209 &  0.959 & -0.153 & -0.045 &  0.093 & -0.227 & -0.143 &  0.101 \\
 Reticon &  47714 &  1989-07-07 &  0.437 &  0.863 & -0.002 & -0.269 &  0.020 &  0.005 &  0.255 &  0.948 & -0.128 & -0.056 &  0.116 & -0.138 & -0.099 &  0.131 \\
 Reticon &  47808 &  1989-10-09 &  0.527 &  1.106 &  0.088 & -0.026 &  0.020 &  0.004 &  0.389 &  1.009 & -0.003 & -0.013 &  0.092 & -0.098